<a href="https://colab.research.google.com/github/jisoolee11/AI_study/blob/main/VGG16_CIFAR10(test1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# 라이브러리 불러오기
import torch
import torchvision
import torchvision.transforms as transforms

In [72]:
# CIFAR10 불러오고 정규화하기
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [73]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision.models import vgg16


class vgg16(nn.Module):
    def __init__(
        self,
        in_channels: int = 3) -> None:
        super().__init__()
        self.conv1_1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(128, 128, kernel_size=1, padding=1)

        self.conv4_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(256, 256, kernel_size=1, padding=1)

        self.conv5_1 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(256, 256, kernel_size=1, padding=1)

        self.pool = nn.AdaptiveAvgPool2d((2, 2))

        self.fc6 = nn.Linear(in_features=256*2*2, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=10)
        
    
    def forward(self, x):
      x = F.relu(self.conv1_1(x))
      x = F.relu(self.conv1_2(x))
      x = self.pool(x)
      x = F.relu(self.conv2_1(x))
      x = F.relu(self.conv2_2(x))
      x = self.pool(x)
      x = F.relu(self.conv3_1(x))
      x = F.relu(self.conv3_2(x))
      x = F.relu(self.conv3_3(x))
      x = self.pool(x)
      x = F.relu(self.conv4_1(x))
      x = F.relu(self.conv4_2(x))
      x = F.relu(self.conv4_3(x))
      x = self.pool(x)
      x = F.relu(self.conv5_1(x))
      x = F.relu(self.conv5_2(x))
      x = F.relu(self.conv5_3(x))
      x = self.pool(x)

      x = x.view(-1, 2*2*256)

      x = F.relu(self.fc6(x))
      x = F.relu(self.fc7(x))
      x = F.relu(self.fc8(x))
      return x

In [75]:
import torch.optim as optim

criterion = nn. CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [76]:
for epoch in range(2):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 2000 == 1999:
      print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.690
[1,  4000] loss: 0.656
[1,  6000] loss: 0.638
[1,  8000] loss: 0.624
[1, 10000] loss: 0.648
[1, 12000] loss: 0.641
[2,  2000] loss: 0.597
[2,  4000] loss: 0.589
[2,  6000] loss: 0.606
[2,  8000] loss: 0.596
[2, 10000] loss: 0.608
[2, 12000] loss: 0.600
Finished Training


In [77]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 70 %
Accuracy of   car : 80 %
Accuracy of  bird : 52 %
Accuracy of   cat : 45 %
Accuracy of  deer : 60 %
Accuracy of   dog : 53 %
Accuracy of  frog : 76 %
Accuracy of horse : 69 %
Accuracy of  ship : 74 %
Accuracy of truck : 76 %
